In [ ]:
!pip install simplejson

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import simplejson

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as fh:
    metadata = json.loads(fh.read())
accountid = metadata['ResourceArn'].split(':')[4]
bucket_name = 'sentiment-analysis-' + accountid
print(bucket_name)

%set_env accountid={accountid}
%set_env bucket_name=sentiment-analysis{accountid}

In [ ]:
%%bash
mkdir analyzed
cd analyzed
aws s3 cp s3://$bucket_name/analyzed_videos/ . --recursive
ls -lah

# Change the name of the video that will ne Analized

The "analyzed/CHANGE-NAME.json" refers to the json generated by the name of the video that were uploaded to the S3.
Change that according to the report that you want to generated.

In [ ]:
# Change the CHANGE-NAME for the name of the video that you uploaded to S3 #
with open("analyzed/CHANGE-NAME.json", "r") as myfile:
    data = myfile.read()
content = json.loads(data)

In [ ]:
df = pd.read_json(data)
df.head()

In [ ]:
df[['SURPRISED', 'HAPPY', 'CALM', 'CONFUSED','SAD', 'FEAR', 'ANGRY', 'DISGUSTED']].max()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=[20, 8])
selected = ['SURPRISED', 'HAPPY', 'CALM', 'CONFUSED','SAD', 'FEAR', 'ANGRY', 'DISGUSTED']
for sentiment in selected:
    plt.plot(df[sentiment], marker='.', linestyle='dashed',
      linewidth=2, markersize=4, label=sentiment)
plt.legend()
plt.title('Rekognition Emotions Analysis')
plt.xlabel('Seconds')
plt.ylabel('Points')

In [ ]:
bp = df[selected].boxplot()
plt.savefig('boxplot.png')

In [ ]:
plt.plot(df.HAPPY, '-')
plt.savefig('HAPPY2.png')

In [ ]:
plt.plot(df.SAD, '-')
plt.savefig('grafico4.png')

In [ ]:
plt.plot(df.CALM, '-')
plt.xlabel('Seconds')
plt.ylabel('Points')
